### 우선 원래 t가 1일 때의 Astute RAG 코드

In [1]:
import warnings
from retrieval import load_vectorstore_retriever_embeddings, rag
from model import llm_load
import json

warnings.filterwarnings("ignore")

# 1. 벡터스토어 로드

# 2. LLM 한 번만 로드 (재사용)
llm = llm_load()

/home/user/anaconda3/envs/knowledge_conflict/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer you are loading from './Mistral-Nemo-Instruct-2407' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Loading weights: 100%|██████████| 363/363 [00:24<00:00, 14.94it/s, Materializing param=model.norm.weight]                               


### 이 중에서 여기까지는 공통

In [2]:
import json
import gc
import torch
import os

from astute_rag import (
    combine_passage, make_internal_passage, make_external_passage, 
    make_prompts, make_passage_source, finalize_answer,
    make_internal_passage_batch, finalize_answer_batch, 
    consolidate_passages, consolidate_passages_batch
)
from prompt_template import P_GEN, P_CON, P_ANS
from langchain_core.prompts import PromptTemplate
from acc_prec import load_results, calculate_accuracy_by_dataset_with_astute_rag

dataset_sizes = {
    "popqa": 260,
    "nq": 260, 
    "triviaqa": 261,
    "bioasq": 261
}

with open("datasets/total_qa_sampled/qa_dataset.json", "r") as f:
    q = json.load(f)

P_gen, P_con, P_ans = make_prompts(P_GEN, P_CON, P_ANS)

E = make_external_passage()

# --- Internal passage: 저장된 파일 있으면 로드, 없으면 생성 후 저장 ---
internal_passage_path = "./output/internal_passages.json"
if os.path.exists(internal_passage_path):
    print(f"Internal passage 로드: {internal_passage_path}")
    with open(internal_passage_path, "r", encoding="utf-8") as f:
        I = json.load(f)
else:
    I = make_internal_passage_batch(q, P_gen, llm, batch_size=4)
    with open(internal_passage_path, "w", encoding="utf-8") as f:
        json.dump(I, f, ensure_ascii=False, indent=2)
    print(f"Internal passage 저장 완료: {internal_passage_path}")

combine_passages = combine_passage(E, I)
passage_source = make_passage_source(combine_passages)

# 컨텍스트 미리 생성
contexts = []
questions = []
for i in range(len(q)):
    context = "\n\n".join([f"[{j+1}]\nsource: {doc['source']}, content: {doc['page_content']}" for j, doc in enumerate(combine_passages[i])])
    contexts.append(context)
    questions.append(q[i]['question'])

# 메모리 정리 후 consolidation 진행
gc.collect()
torch.cuda.empty_cache()



#제발 꺼지지 말아주세요 하나님 부처님 알리님... 벌써 4번 넘게 돌렸는데... 한 번 하는데 5시간은 걸리는데... 젭알 GPU의 신님 제발요제발제발제발제발 제발 내일 돌아왔을 때 모니터 꺼짐 없이 있어주세요 제발요..

Internal passage 로드: ./output/internal_passages.json


In [ ]:
consolidated_contexts = consolidate_passages_batch(
    llm, questions, contexts, t=2, P_con=P_con,
    checkpoint_path="./output/consolidation_checkpoint.json",
    checkpoint_interval=50
)

finalize_answers_path = "./output/output_with_finalize_answers_t2.json"
if os.path.exists(finalize_answers_path):
    print(f"Finalize answers 로드: {finalize_answers_path}")
    with open(finalize_answers_path, "r", encoding="utf-8") as f:
        finalize_answers = json.load(f)
else:
    # contexts = 원본 raw passages (context_init), consolidated_contexts = consolidation 결과 (context)
    finalize_answers = finalize_answer_batch(llm, questions, contexts, batch_size=2, consolidated_contexts=consolidated_contexts)
    with open(finalize_answers_path, "w", encoding="utf-8") as f:
        json.dump(finalize_answers, f, ensure_ascii=False, indent=2)
    print(f"Finalize answers 저장 완료: {finalize_answers_path}")

체크포인트에서 복원: 1042개 완료, 이어서 진행합니다.


consolidating passages: 0it [00:00, ?it/s]

Finalize answers 로드: ./output/output_with_finalize_answers_t2.json


### 설 지나고 이거 하면 됨

In [4]:
results = load_results('./output/output_with_base_api_rag.jsonl')
acc_with_astute_rag = calculate_accuracy_by_dataset_with_astute_rag(results, dataset_sizes, finalize_answers)  
acc_with_astute_rag["overall"] = sum(acc_with_astute_rag.values()) / len(acc_with_astute_rag)

### 밑에거는 경로 코드 무조오오오건  코드 바꿔라

In [5]:
acc_with_astute_rag

{'popqa': 52.7, 'nq': 37.3, 'triviaqa': 87.4, 'bioasq': 54.0, 'overall': 57.85}